In [106]:
import pandas as pd
import numpy as np

In [107]:
df=pd.read_csv('gurgaon_properties_post_feature_selection_.csv')

In [108]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,170.0,3.0,2,2.0,4.0,850.0,0,0,1,1.0,1.0,0.82
1,0.0,235.0,2.0,2,2.0,4.0,1226.0,1,0,1,1.0,2.0,0.95
2,0.0,262.0,2.0,2,1.0,4.0,1000.0,0,0,1,1.0,0.0,0.32
3,0.0,240.0,3.0,4,4.0,2.0,1615.0,1,0,2,0.0,2.0,1.60
4,0.0,127.0,2.0,2,1.0,2.0,582.0,0,1,1,0.0,2.0,0.48


In [109]:
# one hot encoding

In [110]:
x= df.drop(columns=['price'])
y= df['price']

In [111]:
x

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,0.0,170.0,3.0,2,2.0,4.0,850.0,0,0,1,1.0,1.0
1,0.0,235.0,2.0,2,2.0,4.0,1226.0,1,0,1,1.0,2.0
2,0.0,262.0,2.0,2,1.0,4.0,1000.0,0,0,1,1.0,0.0
3,0.0,240.0,3.0,4,4.0,2.0,1615.0,1,0,2,0.0,2.0
4,0.0,127.0,2.0,2,1.0,2.0,582.0,0,1,1,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3700,1.0,134.0,5.0,5,4.0,2.0,6228.0,1,1,1,0.0,1.0
3701,0.0,154.0,1.0,1,1.0,0.0,665.0,0,0,2,2.0,2.0
3702,1.0,15.0,5.0,6,3.0,0.0,5490.0,1,1,1,2.0,2.0
3703,1.0,4.0,1.0,1,0.0,2.0,876.0,0,0,1,1.0,1.0


In [112]:
y

0        0.82
1        0.95
2        0.32
3        1.60
4        0.48
        ...  
3700     6.00
3701     0.60
3702    15.50
3703     0.13
3704     1.78
Name: price, Length: 3705, dtype: float64

In [113]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [114]:
columns_to_encode =['sector','balcony','agepossession','furnishing_type','luxury_category','floor_category']

In [115]:
y_transformed = np.log1p(y)

In [116]:
y_transformed

0       0.598837
1       0.667829
2       0.277632
3       0.955511
4       0.392042
          ...   
3700    1.945910
3701    0.470004
3702    2.803360
3703    0.122218
3704    1.022451
Name: price, Length: 3705, dtype: float64

In [117]:
y_transformed = np.log1p(y)

In [118]:
print(x.isnull().sum())
print(y_transformed.isnull().sum())

property_type      0
sector             0
bedRoom            0
bathroom           0
balcony            0
agePossession      0
built_up_area      0
servant room       0
store room         0
furnishing_type    0
luxury_category    0
floor_category     0
dtype: int64
0


In [120]:

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [121]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [129]:
# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)



In [132]:
scores = cross_val_score(pipeline, x, y_transformed, cv= kfold, scoring='r2')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'agepossession'

The above excepti

In [124]:
print(pipeline.steps)

[('preprocessor', ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['property_type', 'bedRoom', 'bathroom',
                                  'built_up_area', 'servant room',
                                  'store room']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['sector', 'balcony', 'agepossession',
                                  'furnishing_type', 'luxury_category',
                                  'floor_category'])])), ('regressor', SVR())]


In [ ]:
scores.mean()

In [128]:
columns_to_encode = [col for col in columns_to_encode if col in x.columns]
columns_to_encode

['sector', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']

In [ ]:
print(x.columns)  # Check columns in x
print(y_transformed.columns)  # Check columns in y_transformed

In [131]:
print("DataFrame columns:", df.columns.tolist())
print("Features (x) columns:", x.columns.tolist())

DataFrame columns: ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony', 'agePossession', 'built_up_area', 'servant room', 'store room', 'furnishing_type', 'luxury_category', 'floor_category', 'price']
Features (x) columns: ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony', 'agePossession', 'built_up_area', 'servant room', 'store room', 'furnishing_type', 'luxury_category', 'floor_category']


In [135]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Example DataFrame
data = pd.read_csv('gurgaon_properties_post_feature_selection_.csv')

df = pd.DataFrame(data)

# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()

# Define features (x) and target (y)
x = df.drop(columns=['price'])
y = df['price']

# Transform the target variable
y_transformed = np.log1p(y)

# Columns to encode
columns_to_encode = ['sector', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']

# Check for missing columns
missing_columns = set(columns_to_encode) - set(x.columns)
if missing_columns:
    raise ValueError(f"The following columns are missing in the DataFrame: {missing_columns}")

# Preprocessing for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), columns_to_encode),  # Encode categorical columns
        ('num', StandardScaler(), x.columns.difference(columns_to_encode))  # Scale numerical columns
    ]
)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocess the data
    ('model', SVR(kernel='rbf'))    # Use Linear Regression
])

# Define the KFold cross-validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')

# Print the results
print("R2 Scores:", scores)
print("Mean R2 Score:", scores.mean())

C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_response.p

R2 Scores: [nan nan nan nan nan]
Mean R2 Score: nan


C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kishlay Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_response.p